In [ ]:
%pip install sqlalchemy

In [ ]:
"""
Objetos do sqlalchemy core para a realização da conexão e operação do Banco de Dados
"""
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

DB_CONNECTION = URL.create(
    drivername="postgresql",
    username="postgres",
    password="1234",
    host="10.8.0.13",
    port=5432,
    database="stemis"
)
engine = create_engine(DB_CONNECTION, echo=False)
Base = declarative_base()
Session = sessionmaker(bind=engine)

In [ ]:
from sqlalchemy import Column, Integer, String, Boolean, Float, DateTime

class Tcu(Base):
    __tablename__ = "tcu"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    active = Column(Boolean)
    latitude = Column(Float)
    longitude = Column(Float) 

    def get_attr_dict(self):
        return {
            "id": self.id,
            "name": self.name,
            "active": self.active,
            "latitude": self.latitude,
            "longitude": self.longitude
        }


class TcuData(Base):
    __tablename__ = "tcu_data"
    id = Column(Integer, primary_key=True)  
    tcu_id = Column(String) 
    datetime = Column(DateTime)
    tcu_datetime = Column(DateTime)
    angular_position = Column(Float)
    target_angle = Column(Float)

    def get_attr_dict(self):
        return {
            "id": self.id,
            "tcu_id": self.tcu_id,
            "datetime": self.datetime.strftime("%Y-%m-%d %H:%M:%S.%f"),
            "tcu_datetime": self.tcu_datetime.strftime("%Y-%m-%d %H:%M:%S.%f"),
            "angular_position": self.angular_position,
            "target_angle": self.target_angle
        }


In [ ]:
class Usina(object):
    def __init__(self):
        self._session = Session()

    def get_tcus(self):
        tcus = self._session.query(Tcu).all()
        data = [tcu.get_attr_dict() for tcu in tcus]
        return data
    

    def get_tcu_uplinks(self, dev_eui, n):
        uplinks = self._session.query(TcuData).filter(TcuData.tcu_id == dev_eui).order_by(TcuData.datetime.desc()).limit(n).all()
        data = [uplink.get_attr_dict() for uplink in uplinks]
        return data


In [ ]:
usina = Usina()
tcus = usina.get_tcus()
tcus

In [1]:
import time
import struct

timestamp = float(int(time.time())) 
hex_representation = struct.unpack(">I", struct.pack(">f", timestamp))[0]  # Converte para IEEE 754 e extrai como inteiro
print(f"Timestamp: {timestamp}")
print(f"Hex: 0x{hex_representation:08x}")


Timestamp: 1741785487.0
Hex: 0x4ecfa313


In [ ]:
import time
import struct

timestamp = float(int(time.time()))                 
hex_representation = struct.unpack(">I", struct.pack(">f", timestamp))[0]
print(f"COMANDO RTC:")
print(f"e1{hex_representation:08x}")

COMANDO RTC:
Hex: e14ecfa0bb


In [ ]:
import struct
import binascii

def unpack_float(payload, start_idx, length=32):
    return struct.unpack("f", struct.pack("I", int(payload[start_idx:start_idx + length], 2)))[0]

def unpack_int(payload, start_idx, length=16):
    return struct.unpack("<I", struct.pack("I", int(payload[start_idx:start_idx + length], 2)))[0]

def decode_fport2_payload(payload_hex):
    binary_payload = bin(int(payload_hex, 16))[2:].zfill(len(payload_hex) * 4)
    
    decoded_data = {
        "spa.latitude": unpack_float(binary_payload, 0),
        "spa.longitude": unpack_float(binary_payload, 32),
        "spa.elevation": unpack_float(binary_payload, 64),
        "spa.site_pressure": unpack_float(binary_payload, 96),
        "spa.site_temp": unpack_float(binary_payload, 128),
        "spa.surface_azimuth": unpack_float(binary_payload, 160),
        "spa.tracking_max_angle": unpack_float(binary_payload, 192),
        "spa.tracking_min_angle": unpack_float(binary_payload, 224),
        "spa.homePosition": unpack_float(binary_payload, 256),
        "safePositions[0]": unpack_float(binary_payload, 288),
        "spa.table_width": unpack_float(binary_payload, 320),
        "spa.ptp_spacing": unpack_float(binary_payload, 352),
        "inclinometer_offset": unpack_float(binary_payload, 384),
        "spa.slope": unpack_float(binary_payload, 416),
        "spa.tracker_yaw_angle": unpack_float(binary_payload, 448),
        "commPeriod": unpack_int(binary_payload, 480, 16),
        "debugPeriod": unpack_int(binary_payload, 496, 16),
        "lastcommPeriod": unpack_int(binary_payload, 512, 16),
        "movePeriod": unpack_int(binary_payload, 528, 16),
    }
    return decoded_data




In [ ]:
# Payload de exemplo
payload_hex = "c1a625dac246357f443b8000447cc00041c8000000000000425c0000c25c0000c1c80000c1c800004018a3d740d00000be59eae50000000000000000012c0e100708003c"

decoded_payload = decode_fport2_payload(payload_hex)

for key, value in decoded_payload.items():
    print(f"{key}: {value}")

spa.latitude: -20.913087844848633
spa.longitude: -48.46515655517578
spa.elevation: 556.0
spa.site_pressure: 1013.0
spa.site_temp: 25.0
spa.surface_azimuth: 0.0
spa.tracking_max_angle: 55.0
spa.tracking_min_angle: -55.0
spa.homePosition: -25.0
safePositions[0]: -25.0
spa.table_width: 2.384999990463257
spa.ptp_spacing: 5.5
inclinometer_offset: -1.9760215282440186
spa.slope: 0.0
spa.tracker_yaw_angle: 0.0
commPeriod: 300
debugPeriod: 65535
lastcommPeriod: 65535
movePeriod: 65535
